# Make the rd file for the SXDS example table

This is an early test table to include the gen 2 p2021.1 run on the Soton VO.


We need to create all the column descriptors and set the table name and other features.

In [48]:
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
import numpy as np

In [3]:
!ls -al ../../lsst-ir-fusion/dmu5/dmu5_VIDEO/data

total 34456128
drwxr-xr-x@ 22 raphaelshirley  staff         704 21 Jul 15:44 .
drwxr-xr-x@ 21 raphaelshirley  staff         672 25 Jul 11:15 ..
-rw-r--r--   1 raphaelshirley  staff     7654209 12 Apr 17:35 3.0_startCoadd.slurm.e58711243
drwxr-sr-x   6 raphaelshirley  staff         192  6 Apr 18:38 39
drwxr-sr-x   6 raphaelshirley  staff         192  6 Apr 18:29 48
drwxr-sr-x   3 raphaelshirley  staff          96  8 Apr 14:57 66
-rw-r--r--@  1 raphaelshirley  staff    27020160  1 Jul  2021 8523_5,2_reducedCat.fits
-rw-r--r--@  1 raphaelshirley  staff   144984960  4 May 19:13 VSA_Ks_6.fits
-rw-r--r--@  1 raphaelshirley  staff   127722240  1 Jul  2021 calexp-HSC-R-8524-2,7.fits
-rw-r--r--@  1 raphaelshirley  staff   359714880  1 Jul  2021 calexp-VISTA-Y-8524-2,7.fits
-rw-r--r--@  1 raphaelshirley  staff    56338560  5 May 17:42 deepCoadd_calexp_8524_48_K_hscPdr2_u_ir-shir1_DRP_videoCoaddDetect_20220405T133706Z.fits
-rw-r--r--@  1 raphaelshirley  staff   142960320  5 May 17:53 deepCoadd_ca

In [6]:
!mkdir descriptors

In [14]:
t=Table.read('../../lsst-ir-fusion/dmu5/dmu5_VIDEO/data/full_reduced_cat_SXDS_p2021_1_20210702.fits')

In [15]:
len(t)

3119931

In [51]:
t[:5]

id,VISTA_Z_m_slot_ModelFlux_magErr,VISTA_Z_m_slot_ModelFlux_mag,VISTA_Z_m_slot_ModelFlux_fluxErr,VISTA_Z_m_slot_ModelFlux_flux,VISTA_Z_m_slot_ModelFlux_flag,VISTA_Z_m_base_PsfFlux_magErr,VISTA_Z_m_base_PsfFlux_mag,VISTA_Z_m_base_PsfFlux_fluxErr,VISTA_Z_m_base_PsfFlux_flux,VISTA_Z_m_base_PsfFlux_flag,VISTA_Z_m_base_CircularApertureFlux_6_0_magErr,VISTA_Z_m_base_CircularApertureFlux_6_0_mag,VISTA_Z_m_base_CircularApertureFlux_6_0_fluxErr,VISTA_Z_m_base_CircularApertureFlux_6_0_flux,VISTA_Z_m_base_CircularApertureFlux_6_0_flag,VISTA_Y_m_slot_ModelFlux_magErr,VISTA_Y_m_slot_ModelFlux_mag,VISTA_Y_m_slot_ModelFlux_fluxErr,VISTA_Y_m_slot_ModelFlux_flux,VISTA_Y_m_slot_ModelFlux_flag,VISTA_Y_m_base_PsfFlux_magErr,VISTA_Y_m_base_PsfFlux_mag,VISTA_Y_m_base_PsfFlux_fluxErr,VISTA_Y_m_base_PsfFlux_flux,VISTA_Y_m_base_PsfFlux_flag,VISTA_Y_m_base_CircularApertureFlux_6_0_magErr,VISTA_Y_m_base_CircularApertureFlux_6_0_mag,VISTA_Y_m_base_CircularApertureFlux_6_0_fluxErr,VISTA_Y_m_base_CircularApertureFlux_6_0_flux,VISTA_Y_m_base_CircularApertureFlux_6_0_flag,VISTA_Ks_m_slot_ModelFlux_magErr,VISTA_Ks_m_slot_ModelFlux_mag,VISTA_Ks_m_slot_ModelFlux_fluxErr,VISTA_Ks_m_slot_ModelFlux_flux,VISTA_Ks_m_slot_ModelFlux_flag,VISTA_Ks_m_merge_peak_sky,VISTA_Ks_m_detect_isTractInner,VISTA_Ks_m_detect_isPrimary,VISTA_Ks_m_detect_isPatchInner,VISTA_Ks_m_deblend_nChild,VISTA_Ks_m_coord_ra,VISTA_Ks_m_coord_dec,VISTA_Ks_m_base_PsfFlux_magErr,VISTA_Ks_m_base_PsfFlux_mag,VISTA_Ks_m_base_PsfFlux_fluxErr,VISTA_Ks_m_base_PsfFlux_flux,VISTA_Ks_m_base_PsfFlux_flag,VISTA_Ks_m_base_ClassificationExtendedness_value,VISTA_Ks_m_base_ClassificationExtendedness_flag,VISTA_Ks_m_base_CircularApertureFlux_6_0_magErr,VISTA_Ks_m_base_CircularApertureFlux_6_0_mag,VISTA_Ks_m_base_CircularApertureFlux_6_0_fluxErr,VISTA_Ks_m_base_CircularApertureFlux_6_0_flux,VISTA_Ks_m_base_CircularApertureFlux_6_0_flag,VISTA_J_m_slot_ModelFlux_magErr,VISTA_J_m_slot_ModelFlux_mag,VISTA_J_m_slot_ModelFlux_fluxErr,VISTA_J_m_slot_ModelFlux_flux,VISTA_J_m_slot_ModelFlux_flag,VISTA_J_m_base_PsfFlux_magErr,VISTA_J_m_base_PsfFlux_mag,VISTA_J_m_base_PsfFlux_fluxErr,VISTA_J_m_base_PsfFlux_flux,VISTA_J_m_base_PsfFlux_flag,VISTA_J_m_base_CircularApertureFlux_6_0_magErr,VISTA_J_m_base_CircularApertureFlux_6_0_mag,VISTA_J_m_base_CircularApertureFlux_6_0_fluxErr,VISTA_J_m_base_CircularApertureFlux_6_0_flux,VISTA_J_m_base_CircularApertureFlux_6_0_flag,VISTA_H_m_slot_ModelFlux_magErr,VISTA_H_m_slot_ModelFlux_mag,VISTA_H_m_slot_ModelFlux_fluxErr,VISTA_H_m_slot_ModelFlux_flux,VISTA_H_m_slot_ModelFlux_flag,VISTA_H_m_base_PsfFlux_magErr,VISTA_H_m_base_PsfFlux_mag,VISTA_H_m_base_PsfFlux_fluxErr,VISTA_H_m_base_PsfFlux_flux,VISTA_H_m_base_PsfFlux_flag,VISTA_H_m_base_CircularApertureFlux_6_0_magErr,VISTA_H_m_base_CircularApertureFlux_6_0_mag,VISTA_H_m_base_CircularApertureFlux_6_0_fluxErr,VISTA_H_m_base_CircularApertureFlux_6_0_flux,VISTA_H_m_base_CircularApertureFlux_6_0_flag,HSC_Z_m_slot_ModelFlux_magErr,HSC_Z_m_slot_ModelFlux_mag,HSC_Z_m_slot_ModelFlux_fluxErr,HSC_Z_m_slot_ModelFlux_flux,HSC_Z_m_slot_ModelFlux_flag,HSC_Z_m_base_PsfFlux_magErr,HSC_Z_m_base_PsfFlux_mag,HSC_Z_m_base_PsfFlux_fluxErr,HSC_Z_m_base_PsfFlux_flux,HSC_Z_m_base_PsfFlux_flag,HSC_Z_m_base_CircularApertureFlux_6_0_magErr,HSC_Z_m_base_CircularApertureFlux_6_0_mag,HSC_Z_m_base_CircularApertureFlux_6_0_fluxErr,HSC_Z_m_base_CircularApertureFlux_6_0_flux,HSC_Z_m_base_CircularApertureFlux_6_0_flag,HSC_Y_m_slot_ModelFlux_magErr,HSC_Y_m_slot_ModelFlux_mag,HSC_Y_m_slot_ModelFlux_fluxErr,HSC_Y_m_slot_ModelFlux_flux,HSC_Y_m_slot_ModelFlux_flag,HSC_Y_m_base_PsfFlux_magErr,HSC_Y_m_base_PsfFlux_mag,HSC_Y_m_base_PsfFlux_fluxErr,HSC_Y_m_base_PsfFlux_flux,HSC_Y_m_base_PsfFlux_flag,HSC_Y_m_base_CircularApertureFlux_6_0_magErr,HSC_Y_m_base_CircularApertureFlux_6_0_mag,HSC_Y_m_base_CircularApertureFlux_6_0_fluxErr,HSC_Y_m_base_CircularApertureFlux_6_0_flux,HSC_Y_m_base_CircularApertureFlux_6_0_flag,HSC_R_m_slot_ModelFlux_magErr,HSC_R_m_slot_ModelFlux_mag,HSC_R_m_slot_Mod

In [52]:
#convert positions to degrees
t['VISTA_Ks_m_coord_ra']=t['VISTA_Ks_m_coord_ra'].to(u.deg)
t['VISTA_Ks_m_coord_dec']=t['VISTA_Ks_m_coord_dec'].to(u.deg)
#remove 1e+20 values


In [87]:
rd_start="""
<resource schema="lsst_ir_fusion">
    <meta name="title">
        LSST IR fusion catalogue
    </meta>
    <meta name="creationDate">
        {time_now}
    </meta>
    <meta name="description">
        A first gen 2 SXDS run with reduced column set to test the Soton VO.
    </meta>
    <meta name="creator.name">
        Shirley, R.; et al
    </meta>
    <meta name="subject">
        Catalogs
    </meta>
    <meta name="subject">
        Galaxies
    </meta>
    <meta name="coverage">
        <meta name="profile">
            AllSky ICRS
        </meta>
        <meta name="waveband">
            Optical
        </meta>
        <meta name="waveband">
            Infrared
        </meta>
    </meta>
    <meta name="_longdoc" format="rst">
        <![CDATA[
        This resource contains the A list catalogue from the Herschel Extragalactic
        Legacy Project (HELP). These are all objects which have an XID+ flux.

        History
        -------

        ========== ============================================
        {date_now} First SXDS test VO ingestion
        ========== ============================================
        ]]>
    </meta>
    <meta name="source">
    </meta>
    <meta name="_intro" format="rst">
        <![CDATA[
        For advanced queries on this catalogue use ADQL_
        possibly via TAP_

        .. _ADQL: /adql
        .. _TAP: /tap
        ]]>
    </meta>
    <table id="main" onDisk="True" adql="True" mixin="//scs#q3cindex" primary="help_id">
        <stc>
            Position ICRS Epoch J2000.0 "ra" "dec"
        </stc>
        <index columns="id">
            <FEED source="%#fastindex" />
        </index>
        <index columns="ra">
            <FEED source="%#fastindex" />
        </index>
        <index columns="dec">
            <FEED source="%#fastindex" />
        </index>
""".format(time_now=time_now,date_now=time_now[0:10])

col_template="""
        <column 
            name="{name}" 
            tablehead="{name}" 
            type="{dtype}" 
            ucd="{ucd}" 
            unit="{unit}"
            description="{description}" 
            verbLevel="1" 
            />
"""

rd_end="""
<meta name="note" tag="1">
            <![CDATA[
            This table is a prototype and should be treated with caution
            ]]>
        </meta>

    </table>
    <data id="import">
        <sources>
            <pattern>data/main/*.csv</pattern>
        </sources>
        <csvGrammar />
        <make table="main">
            <rowmaker idmaps="*">
                <simplemaps>
                </simplemaps>
            </rowmaker>
        </make>
    </data>
    <service id="cone" allowed="scs.xml,form,static">
        <meta name="title">
            LSST IR fusion SXDS prototype p2021p1
        </meta>
        <meta name="shortName">
            SXDS p2021p1
        </meta>
        <meta name="testQuery">
            <meta name="ra">
                35.0
            </meta>
            <meta name="dec">
                -5.0
            </meta>
            <meta name="sr">
                1.0
            </meta>
        </meta>
        <!--
        this is to allow access to the raw data.  Decide for yourself
        whether or not you want this
        -->
        <property name="staticData">
            data/files
        </property>
        <dbCore queriedTable="main">
            <FEED source="//scs#coreDescs" />
            <condDesc buildFrom="field" />
            <condDesc buildFrom="help_id" />
        </dbCore>
        <publish render="scs.xml" sets="ivo_managed" />
        <publish render="form" sets="ivo_managed,local" />
        <outputTable verbLevel="20" />
    </service>
</resource>
"""

In [92]:
rd_file = open('descriptors/sxds_p2021p1.rd', 'w+')
rd_file.write(rd_start)

for c in t.colnames:
    
    #all big integers except for flags to booleans
    if t[c].dtype in [np.dtype('>i4'),np.dtype('>i8')]:
        dtype='bigint'
    # all singles except for coords
    elif t[c].dtype in [np.dtype('>f8'),np.dtype('float64')]:
        dtype='single precision'
    elif 'coord' in c:
        dtype='double precision'
    elif 'flag' in c:
        dtype='boolean'
        
    # UCDs currently not set fully
    description=t[c].description
    if not description==None:
        description=description.replace('"','')
    
    rd_file.write(col_template.format(
        name=t[c].name,
        dtype=dtype,
        ucd=ucd,
        unit=t[c].unit,
        description=description
    ))
rd_file.write(rd_end)
rd_file.close()

## Save the catalogue as a csv file

We need to make a couple of changes to the file and save it as a csv for ingestion

In [72]:
!mkdir data

mkdir: data: File exists


In [76]:
filename='data/sxds_p2021p1_{}.csv'.format(time_now.replace('-','').replace(':','')[:-4])
print(filename)
t.write(filename,overwrite=True)

data/sxds_p2021p1_20220728T160313.csv
